In [1]:
import numpy as np
import matplotlib.pyplot as plt
from MultinestClusterFit import MultinestClusterFit
from RebinningFunctions import rebin_data_exp, rebin_data_exp_50, spimodfit_binning_SE
from PointingClusters import PointingClusters, save_clusters, load_clusters
from ModelSources import *
import pickle

[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available
[INFO    ] Starting 3ML!
[WARNING ] no display variable set. using backend for graphics without display (agg)
[WARNING ] ROOT minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] No fermitools installed
[WARNING ] Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances 

In [12]:
folder = "sim_source_1380"
pointings = load_clusters(folder)
pointings

((('138000030010', 'crab_data/1380'), ('138000050010', 'crab_data/1380')),
 (('138000040010', 'crab_data/1380'), ('138000060010', 'crab_data/1380')),
 (('138000070010', 'crab_data/1380'), ('138000080010', 'crab_data/1380')),
 (('138000090010', 'crab_data/1380'), ('138000100010', 'crab_data/1380')),
 (('138000110010', 'crab_data/1380'), ('138000120010', 'crab_data/1380')),
 (('138000130010', 'crab_data/1380'), ('138000140010', 'crab_data/1380')),
 (('138000150010', 'crab_data/1380'), ('138000160010', 'crab_data/1380')),
 (('138000170010', 'crab_data/1380'), ('138000180010', 'crab_data/1380')),
 (('138000190010', 'crab_data/1380'), ('138000200010', 'crab_data/1380')),
 (('138000210010', 'crab_data/1380'), ('138000220010', 'crab_data/1380')),
 (('138000240010', 'crab_data/1380'), ('138000250010', 'crab_data/1380')),
 (('138000260010', 'crab_data/1380'), ('138000270010', 'crab_data/1380')),
 (('138000280010', 'crab_data/1380'), ('138000300010', 'crab_data/1380')),
 (('138000290010', 'crab_

In [2]:
data_folder = "simulated_data/0374_const_bkg"

with open(f"./{data_folder}/source_params.pickle", "rb") as f:
    source_ra, source_dec, source_piv, source_Ks, source_indices = pickle.load(f)
    

for i in range(len(source_Ks)):
    for j in range(len(source_indices)):
        
        temp_path = f"{data_folder}/{i}_{j}"

        pointings = PointingClusters(
            (temp_path,),
            min_angle_dif=1.5,
            max_angle_dif=7.5,
            max_time_dif=0.2,
            radius_around_source=10.,
            min_time_elapsed=300.,
            cluster_size_range=(2,2),
            center_ra=10.,
            center_dec=-40.,
        ).pointings
        save_clusters(pointings, temp_path)

        pointings = load_clusters(temp_path)
        source_model = define_sources((
            (simulated_pl_0374, (100,)),
        ))

        multinest_fit = MultinestClusterFit(
            pointings,
            source_model,
            (None, None),
            np.geomspace(18, 3000, 50),
            rebin_data_exp_50,
            # true_values=true_values(),
            folder=temp_path,
        )

        multinest_fit.parameter_fit_distribution()
        multinest_fit.text_summaries(pointing_combinations=True, reference_values=False, parameter_fit_constraints=False)
        # multinest_fit.ppc()
        
        # print(multinest_fit._cc._all_parameters)
        
        p = ["Simulated Source 0374 K", "Simulated Source 0374 index"]
        val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
        cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

        with open(f"./{temp_path}/crab_parameters.pickle", "wb") as f:
            pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.997653
Replacements:                                850
Total Samples:                               852
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999068
Acceptance Rate:                        0.995575
Replacements:                                900
Total Samples:                               904
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999102
Acceptance Rate:                        0.989583
Replacements:                                950
Total Samples:                               960
Nested Sampling

Acceptance Rate:                        0.734702
Replacements:                              12991
Total Samples:                             17682
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.015556
 ln(ev)=   512265505.38283032      +/-                       NaN
 Total Likelihood Evaluations:        17682
 Sampling finished. Exiting MultiNest
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.997653
Replacements:                                850
Total Samples:                               852
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999413
Acceptance Rate

In [2]:
data_folder = "simulated_data/0374_const_bkg_sec_source"

with open(f"./{data_folder}/source_params.pickle", "rb") as f:
    source_ra, source_dec, source_piv, source_K, source_index, s_source_decs, s_source_Ks = pickle.load(f)
    

for i in range(len(s_source_decs)):
    for j in range(len(s_source_Ks)):
        
        temp_path = f"{data_folder}/{i}_{j}"

        pointings = PointingClusters(
            (temp_path,),
            min_angle_dif=1.5,
            max_angle_dif=7.5,
            max_time_dif=0.2,
            radius_around_source=10.,
            min_time_elapsed=300.,
            cluster_size_range=(2,2),
            center_ra=10.,
            center_dec=-40.,
        ).pointings
        save_clusters(pointings, temp_path)

        pointings = load_clusters(temp_path)
        source_model = define_sources((
            (simulated_pl_0374, (100,)),
        ))

        multinest_fit = MultinestClusterFit(
            pointings,
            source_model,
            (None, None),
            np.geomspace(18, 3000, 50),
            rebin_data_exp_50,
            # true_values=true_values(),
            folder=temp_path,
        )

        multinest_fit.parameter_fit_distribution()
        multinest_fit.text_summaries(pointing_combinations=True, reference_values=False, parameter_fit_constraints=False)
        # multinest_fit.ppc()
        
        # print(multinest_fit._cc._all_parameters)
        
        p = ["Simulated Source 0374 K", "Simulated Source 0374 index"]
        val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
        cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

        with open(f"./{temp_path}/crab_parameters.pickle", "wb") as f:
            pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.997653
Replacements:                                850
Total Samples:                               852
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999413
Acceptance Rate:                        0.991189
Replacements:                                900
Total Samples:                               908
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999449
Acceptance Rate:                        0.981405
Replacements:                                950
Total Samples:                               968
Nested Sampling

 ln(ev)=   500708853.10945511      +/-  0.19974274089875479     
 Total Likelihood Evaluations:        16622
 Sampling finished. Exiting MultiNest
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.993377
Replacements:                                900
Total Samples:                               906
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999448
Acceptance Rate

Acceptance Rate:                        0.795213
Replacements:                              11595
Total Samples:                             14581
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.016654
 ln(ev)=   504446894.71955043      +/-   5.2376514115901121E-002
 Total Likelihood Evaluations:        14581
 Sampling finished. Exiting MultiNest
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        1.000000
Replacements:                                850
Total Samples:                               850
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate

ERROR: Interrupt received: Terminating
Exception ignored on calling ctypes callback function: <function run.<locals>.loglike at 0x7fdab4d3faf0>
Traceback (most recent call last):
  File "/home/moej56153/.pyenv/versions/MT/lib/python3.9/site-packages/pymultinest/run.py", line 221, in loglike
    return LogLikelihood(cube, ndim, nparams)
  File "/home/moej56153/Master_Thesis/integral_data_analysis/MultinestClusterFit.py", line 253, in logLba_mult
    return logLcore(
  File "/home/moej56153/.pyenv/versions/MT/lib/python3.9/site-packages/pymultinest/run.py", line 70, in interrupt_handler
    sys.exit(1)
SystemExit: 1


Acceptance Rate:                        0.776053
Replacements:                               8400
Total Samples:                             10824
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.111118
Acceptance Rate:                        0.776155
Replacements:                               8450
Total Samples:                             10887
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.107740
Acceptance Rate:                        0.776539
Replacements:                               8500
Total Samples:                             10946
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.104922
Acceptance Rate:                        0.776990
Replacements:                               8550
Total Samples:                             11004
Nested Sampling ln(Z):            **************
Importance Nested Sampling 

In [6]:
folder = "test_ba_de_so_in"

pointings = PointingClusters(
    ("simulated_data/test_ba_de_so_in",),
    min_angle_dif=1.5,
    max_angle_dif=7.5,
    max_time_dif=0.2,
    radius_around_source=10.,
    min_time_elapsed=300.,
    cluster_size_range=(2,2),
    center_ra=10.,
    center_dec=-40.,
).pointings
save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (simulated_pl_0374, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (None, None),
    np.geomspace(18, 3000, 200),
    rebin_data_exp_50,
    # true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries(pointing_combinations=True, reference_values=False, parameter_fit_constraints=False)
# multinest_fit.ppc()

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.997653
Replacements:                                850
Total Samples:                               852
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999413
Acceptance Rate:                        0.990099
Replacements:                                900
Total Samples:                               909
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999450
Acceptance Rate:                        0.980392
Replacements:                                950
Total Samples:                               969
Nested Sampling

In [8]:
folder = "sim_source_0374"

pointings = PointingClusters(
    ("crab_data/0374",),
    min_angle_dif=1.5,
    max_angle_dif=7.5,
    max_time_dif=0.2,
    radius_around_source=10.,
    min_time_elapsed=300.,
    cluster_size_range=(2,2),
    center_ra=10.,
    center_dec=-40.,
).pointings
save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (simulated_pl_0374, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_50,
    # true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries(pointing_combinations=True, reference_values=False, parameter_fit_constraints=False)
# multinest_fit.ppc()

import pickle
p = ["Simulated Source 0374 K", "Simulated Source 0374 index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)


 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        1.000000
Replacements:                                850
Total Samples:                               850
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.989011
Replacements:                                900
Total Samples:                               910
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999450
Acceptance Rate:                        0.978373
Replacements:                                950
Total Samples:                               971
Nested Sampling

In [13]:
folder = "sim_source_0374_w_smf_bkg"

pointings = PointingClusters(
    ("crab_data/0374_spimodfit_bkg",),
    min_angle_dif=1.5,
    max_angle_dif=7.5,
    max_time_dif=0.2,
    radius_around_source=10.,
    min_time_elapsed=300.,
    cluster_size_range=(2,2),
    center_ra=10.,
    center_dec=-40.,
).pointings
save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (simulated_pl_0374, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    spimodfit_binning_SE,
    # true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries(pointing_combinations=True, reference_values=False, parameter_fit_constraints=False)
# multinest_fit.ppc()

import pickle
p = ["Simulated Source 0374 K", "Simulated Source 0374 index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.997653
Replacements:                                850
Total Samples:                               852
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999413
Acceptance Rate:                        0.993377
Replacements:                                900
Total Samples:                               906
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999448
Acceptance Rate:                        0.988554
Replacements:                                950
Total Samples:                               961
Nested Sampling

In [7]:
folder = "sim_source_0374_const_bkg"

pointings = PointingClusters(
    ("crab_data/0374_const_bkg",),
    min_angle_dif=1.5,
    max_angle_dif=7.5,
    max_time_dif=0.2,
    radius_around_source=10.,
    min_time_elapsed=300.,
    cluster_size_range=(2,2),
    center_ra=10.,
    center_dec=-40.,
).pointings
save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (simulated_pl_0374, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_50,
    # true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries(pointing_combinations=True, reference_values=False, parameter_fit_constraints=False)
# multinest_fit.ppc()

import pickle
p = ["Simulated Source 0374 K", "Simulated Source 0374 index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.996678
Replacements:                                900
Total Samples:                               903
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999446
Acceptance Rate:                        0.986501
Replacements:                                950
Total Samples:                               963
Nested Sampling

In [9]:
folder = "sim_source_1380"

pointings = PointingClusters(
    ("crab_data/1380",),
    min_angle_dif=1.5,
    max_angle_dif=7.5,
    max_time_dif=0.2,
    radius_around_source=10.,
    min_time_elapsed=300.,
    cluster_size_range=(2,2),
    center_ra=155.,
    center_dec=75.,
).pointings
save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (simulated_pl_1380, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_50,
    # true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries(pointing_combinations=True, reference_values=False, parameter_fit_constraints=False)
# multinest_fit.ppc()

import pickle
p = ["Simulated Source 1380 K", "Simulated Source 1380 index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.989011
Replacements:                                900
Total Samples:                               910
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.934359
Acceptance Rate:                        0.981405
Replacements:                                950
Total Samples:                               968
Nested Sampling

In [10]:
folder = "sim_source_1380_w_smf_bkg"

pointings = PointingClusters(
    ("crab_data/1380_spimodfit_bkg",),
    min_angle_dif=1.5,
    max_angle_dif=7.5,
    max_time_dif=0.2,
    radius_around_source=10.,
    min_time_elapsed=300.,
    cluster_size_range=(2,2),
    center_ra=155.,
    center_dec=75.,
).pointings
save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (simulated_pl_1380, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    spimodfit_binning_SE,
    # true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries(pointing_combinations=True, reference_values=False, parameter_fit_constraints=False)
# multinest_fit.ppc()

import pickle
p = ["Simulated Source 1380 K", "Simulated Source 1380 index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.992282
Replacements:                                900
Total Samples:                               907
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999449
Acceptance Rate:                        0.984456
Replacements:                                950
Total Samples:                               965
Nested Sampling

In [15]:
folder = "sim_source_1380_const_bkg"

pointings = PointingClusters(
    ("crab_data/1380_const_bkg",),
    min_angle_dif=1.5,
    max_angle_dif=7.5,
    max_time_dif=0.2,
    radius_around_source=10.,
    min_time_elapsed=300.,
    cluster_size_range=(2,2),
    center_ra=155.,
    center_dec=75.,
).pointings
save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (simulated_pl_1380, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    spimodfit_binning_SE,
    # true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries(pointing_combinations=True, reference_values=False, parameter_fit_constraints=False)
# multinest_fit.ppc()

import pickle
p = ["Simulated Source 1380 K", "Simulated Source 1380 index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.997653
Replacements:                                850
Total Samples:                               852
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999413
Acceptance Rate:                        0.994475
Replacements:                                900
Total Samples:                               905
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999447
Acceptance Rate:                        0.988554
Replacements:                                950
Total Samples:                               961
Nested Sampling

In [5]:
pointings = load_clusters(folder)
for p in pointings:
    print(p)

(('037400020010', 'crab_data/0374'), ('037400030010', 'crab_data/0374'))
(('037400040010', 'crab_data/0374'), ('037400050010', 'crab_data/0374'))
(('037400060010', 'crab_data/0374'), ('037400070010', 'crab_data/0374'))
(('037400080010', 'crab_data/0374'), ('037400090010', 'crab_data/0374'))
(('037400100010', 'crab_data/0374'), ('037400110010', 'crab_data/0374'))
(('037400120010', 'crab_data/0374'), ('037400130010', 'crab_data/0374'))
(('037400140010', 'crab_data/0374'), ('037400150010', 'crab_data/0374'))
(('037400160010', 'crab_data/0374'), ('037400170010', 'crab_data/0374'))
(('037400180010', 'crab_data/0374'), ('037400190010', 'crab_data/0374'))
(('037400210010', 'crab_data/0374'), ('037400220010', 'crab_data/0374'))
(('037400230010', 'crab_data/0374'), ('037400240010', 'crab_data/0374'))
(('037400250010', 'crab_data/0374'), ('037400260010', 'crab_data/0374'))
(('037400270010', 'crab_data/0374'), ('037400280010', 'crab_data/0374'))
(('037400290010', 'crab_data/0374'), ('037400300010

In [3]:
folder = "orbit_1644"

# pointings = PointingClusters(
#     ("crab_data/1644",),
#     min_angle_dif=1.5,
#     max_angle_dif=7.5,
#     max_time_dif=0.2,
#     radius_around_crab=10.,
#     min_time_elapsed=600.,
#     cluster_size_range=(2,2),
# ).pointings
# save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_50,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

FileNotFoundError: [Errno 2] No such file or directory: 'crab_data/1644/pointing.fits'

In [2]:
folder = "orbit_1644_wo_outliers"

folder2 = "orbit_1644"
pointings2 = load_clusters(folder2)

pointings = []

bad_pointings = (
    "164400040010",
    "164400050010",
    "164400060010",
    "164400070010",
    "164400080010",
    "164400090010",
)
for cluster in pointings2:
    if cluster[0][0] in bad_pointings:
        continue
    else:
        pointings.append(cluster)
        
pointings = tuple(pointings)

source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.993377
Replacements:                                900
Total Samples:                               906
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999448
Acceptance Rate:                        0.988554
Replacements:                                950
Total Samples:                               961
Nested Sampling

In [3]:
folder = "orbit_1658"

# pointings = PointingClusters(
#     ("crab_data/1658",),
#     min_angle_dif=1.5,
#     max_angle_dif=7.5,
#     max_time_dif=0.2,
#     radius_around_crab=10.,
#     min_time_elapsed=600.,
#     cluster_size_range=(2,2),
# ).pointings
# save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_50,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
# multinest_fit.ppc()


# import pickle
# p = ["Crab K", "Crab index"]
# val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
# cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

# with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
#     pickle.dump((val, cov), f)

[[0.00376778 0.00360822 0.00345542 0.00330909 0.00316896 0.00303476
  0.00290624 0.00278317 0.00266531 0.00255244 0.00244435 0.00234084
  0.00224171 0.00214678 0.00205586 0.0019688  0.00188543 0.00180558
  0.00172912 0.0016559  0.00158577 0.00151862 0.00145431 0.00139272
  0.00133374 0.00127726 0.00122317 0.00117137 0.00112177 0.00107426
  0.00102877 0.00098521 0.00094348 0.00090353 0.00086527 0.00082862
  0.00079353 0.00075993 0.00072775 0.00069693 0.00066742 0.00063915
  0.00061209 0.00058617 0.00056134 0.00053757 0.00051481 0.000493
  0.00047213]]
float64
((('165800020010', 'crab_data/1658'), ('165800030010', 'crab_data/1658')), (('165800080010', 'crab_data/1658'), ('165800090010', 'crab_data/1658')), (('165800100010', 'crab_data/1658'), ('165800110010', 'crab_data/1658')), (('165800160010', 'crab_data/1658'), ('165800170010', 'crab_data/1658')), (('165800180010', 'crab_data/1658'), ('165800200010', 'crab_data/1658')), (('165800060010', 'crab_data/1658'), ('165800070010', 'crab_data

In [6]:
folder = "orbit_1661"

# pointings = PointingClusters(
#     ("crab_data/1661",),
#     min_angle_dif=1.5,
#     max_angle_dif=7.5,
#     max_time_dif=0.2,
#     radius_around_crab=10.,
#     min_time_elapsed=600.,
#     cluster_size_range=(2,2),
# ).pointings
# save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.994475
Replacements:                                900
Total Samples:                               905
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999447
Acceptance Rate:                        0.981405
Replacements:                                950
Total Samples:                               968
Nested Sampling

In [3]:
folder = "orbit_1661_wo_outliers"

folder2 = "orbit_1661"
pointings2 = load_clusters(folder2)

pointings = []

bad_pointings = (
    "166100020010",
    "166100030010",
    "166100240010",
    "166100260010",
    "166100270010",
    "166100290010",
    "166100280010",
    "166100300010",
    "166100430010",
    "166100440010",
    "166100500010",
    "166100510010",
)
for cluster in pointings2:
    if cluster[0][0] in bad_pointings:
        continue
    else:
        pointings.append(cluster)
        
pointings = tuple(pointings)

source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.995575
Replacements:                                900
Total Samples:                               904
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999447
Acceptance Rate:                        0.988554
Replacements:                                950
Total Samples:                               961
Nested Sampling

In [7]:
folder = "orbit_1662"

# pointings = PointingClusters(
#     ("crab_data/1662",),
#     min_angle_dif=1.5,
#     max_angle_dif=7.5,
#     max_time_dif=0.2,
#     radius_around_crab=10.,
#     min_time_elapsed=600.,
#     cluster_size_range=(2,2),
# ).pointings
# save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.992282
Replacements:                                900
Total Samples:                               907
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999449
Acceptance Rate:                        0.982420
Replacements:                                950
Total Samples:                               967
Nested Sampling

In [4]:
folder = "orbit_1662_wo_outliers"

folder2 = "orbit_1662"
pointings2 = load_clusters(folder2)

pointings = []

bad_pointings = (
    "166200030010",
    "166200040010",
    "166200270010",
    "166200290010",
    "166200450010",
    "166200460010",
    "166200470010",
    "166200520010",
    "166200480010",
    "166200510010",
    "166200490010",
    "166200500010",
    "166200490020",
    "166200540010",
    "166200550010",
    "166200560010",
)
for cluster in pointings2:
    if cluster[0][0] in bad_pointings:
        continue
    else:
        pointings.append(cluster)
        
pointings = tuple(pointings)

source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.992282
Replacements:                                900
Total Samples:                               907
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999449
Acceptance Rate:                        0.986501
Replacements:                                950
Total Samples:                               963
Nested Sampling

In [3]:
folder = "orbit_1664"

# pointings = PointingClusters(
#     ("crab_data/1664",),
#     min_angle_dif=1.5,
#     max_angle_dif=7.5,
#     max_time_dif=0.2,
#     radius_around_crab=10.,
#     min_time_elapsed=600.,
#     cluster_size_range=(2,2),
# ).pointings
# save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.995316
Replacements:                                850
Total Samples:                               854
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999414
Acceptance Rate:                        0.990099
Replacements:                                900
Total Samples:                               909
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999450
Acceptance Rate:                        0.985477
Replacements:                                950
Total Samples:                               964
Nested Sampling

In [5]:
folder = "orbit_1664_wo_outliers"

folder2 = "orbit_1664"
pointings2 = load_clusters(folder2)

pointings = []

bad_pointings = (
    "166400170010",
    "166400180010",
)
for cluster in pointings2:
    if cluster[0][0] in bad_pointings:
        continue
    else:
        pointings.append(cluster)
        
pointings = tuple(pointings)

source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.996483
Replacements:                                850
Total Samples:                               853
Nested Sampling ln(Z):            1352161.921169
Importance Nested Sampling ln(Z): ************** +/-  0.999414
Acceptance Rate:                        0.992282
Replacements:                                900
Total Samples:                               907
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999449
Acceptance Rate:                        0.984456
Replacements:                                950
Total Samples:                               965
Nested Sampling

In [2]:
folder = "orbit_1667"

# pointings = PointingClusters(
#     ("crab_data/1667",),
#     min_angle_dif=1.5,
#     max_angle_dif=7.5,
#     max_time_dif=0.2,
#     radius_around_crab=10.,
#     min_time_elapsed=600.,
#     cluster_size_range=(2,2),
# ).pointings
# save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.992991
Replacements:                                850
Total Samples:                               856
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999416
Acceptance Rate:                        0.987925
Replacements:                                900
Total Samples:                               911
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999451
Acceptance Rate:                        0.979381
Replacements:                                950
Total Samples:                               970
Nested Sampling

In [2]:
folder = "orbit_1667_spimodfit_scws"

pointings = load_clusters(folder)
source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.998825
Replacements:                                850
Total Samples:                               851
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999412
Acceptance Rate:                        0.992282
Replacements:                                900
Total Samples:                               907
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999449
Acceptance Rate:                        0.983437
Replacements:                                950
Total Samples:                               966
Nested Sampling

In [4]:
folder = "orbit_1667_spimodfit_wo_outliers"

folder2 = "orbit_1667_spimodfit_scws"
pointings2 = load_clusters(folder2)

pointings = []

bad_pointings = (
    "166700470010",
    "166700480010",
    "166700500010",
    "166700500020",
    "166700520010",
    "166700530010",
    "166700540010",
    "166700550010",
)
for cluster in pointings2:
    if cluster[0][0] in bad_pointings:
        continue
    else:
        pointings.append(cluster)
        
pointings = tuple(pointings)

source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp_10,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc()


import pickle
p = ["Crab K", "Crab index"]
val = np.array([i[1] for i in multinest_fit._cc.analysis.get_summary(parameters=p).values()])
cov = multinest_fit._cc.analysis.get_covariance(parameters=p)[1]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov), f)

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  800
 dimensionality =    2
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.996483
Replacements:                                850
Total Samples:                               853
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999414
Acceptance Rate:                        0.992282
Replacements:                                900
Total Samples:                               907
Nested Sampling ln(Z):            **************
Importance Nested Sampling ln(Z): ************** +/-  0.999449
Acceptance Rate:                        0.984456
Replacements:                                950
Total Samples:                               965
Nested Sampling